In [1]:
# Constants
DF_PATH = "../data/processed/1_preprocessed_df.pkl"

NA_STRING = 'Not Specified'
TRANSPARENT_STRING = 'rgba(0, 0, 0)'

ROLE_COLS       = ['DevType']
TECH_COLS       = ['LanguageHaveWorkedWith',
                   'DatabaseHaveWorkedWith',
                   'PlatformHaveWorkedWith',
                   'WebframeHaveWorkedWith',
                   'MiscTechHaveWorkedWith',
                   'ToolsTechHaveWorkedWith',
                   'NEWCollabToolsHaveWorkedWith']
EXPORT_FEATURES_DIR = "../data/processed"

In [9]:
# Load packages
import pandas as pd
import numpy as np
import logging
import pickle
import os

import plotly
import plotly.express as px
import plotly.graph_objects as go # Graph objects are the low-level building blocks of figures which you can use instead of Plotly Express for greater control.

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier #, MultiOutputRegressor
from sklearn.metrics import multilabel_confusion_matrix

from sklearn.pipeline import make_pipeline
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split

pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000

### Read data and preprocess data

In [3]:
# Read Data 
df = pd.read_pickle(DF_PATH)

In [4]:
# One hot encode
encoded_dfs = {}
for col in ROLE_COLS + TECH_COLS :
    binarizer = MultiLabelBinarizer()
    encoded_df = pd.DataFrame(binarizer.fit_transform(df[col]),
                              columns = binarizer.classes_,
                              index= df[col].index)
    encoded_dfs[col] = encoded_df

# Merge 1-hot encoded
ohe_df = pd.concat(encoded_dfs, axis=1)

### Dimentionality reduction

In [5]:
# Prepare sub frames
skills_ohe = ohe_df.drop('DevType', axis=1).copy()
std_skills = StandardScaler().fit_transform(skills_ohe)

/Users/ahmadsamir/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/Users/ahmadsamir/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [6]:
tsne_projection = TSNE(n_components=2,
                       perplexity=3,
                       learning_rate=0.01,
                       init='pca',
                       method='barnes_hut',
                       n_jobs=2,
                       n_iter=10**10,
                       random_state=0).fit_transform(std_skills)

tsne_projection = pd.DataFrame(tsne_projection, index=skills_ohe.columns)


KeyboardInterrupt: 

In [ ]:
tsne_projection.shape

In [ ]:
fig = px.scatter(x=tsne_projection[0],
                 y=tsne_projection[1],
                 text=tsne_projection.droplevel(0).index)
fig.update_traces(texposition='top center')
fig.update_layout(height=1000, width=1000, title_text='TSNE')
fig.show()

### Cluster

In [6]:
range_n_cluster = list(range(10,25))
silhoutte_score = []
best_cluster_model = None

for n_clusters in range_n_cluster:
    cluster_model = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    cluster_labels = cluster_model.fit_predict(tsne_projection)
    
    silhoutte_avg = silhoutte_score(tsne_projection, cluster_labels)
    silhoutte_score += [silhoutte_avg]
    
    if silhoutte_avg >= np.max(silhoutte_score):
        best_cluster_model = cluster_model



NameError: name 'tsne_projection' is not defined

In [ ]:
plt.plot(range_n_cluster, silhoutte_score)
plt.axvline(best_cluster_model.n_clusters, color='black')

In [ ]:
cluster_labels = ["skills_group_" + str(label)
                  for label in best_cluster_model.labels_]

In [ ]:
fig = px.scatter(x=tsne_projection[0],
                 y=tsne_projection[1],
                 text=tsne_projection.droplevel(0).index,
                 color=cluster_labels)
fig.update_layout(height=800, width=800, title_text='Cluster')
fig.show()

In [ ]:
# How to better write this
skills_clusters = tsne_projection.index.droplevel(0).to_series().groupby(cluster_labels).apply(list)

In [ ]:
for cluster, skills in skills_clusters.items():
    print(cluster)
    print(skills)

### Create new features

In [ ]:
new_features = []

for cluster, skills in skills_clusters.items():
    cluster_sum = skill_ohe.droplevel(0, axis=1)[skills].sum(axis=1)
    cluster_sum.name = cluster
    new_features.append(cluster_sum)

fe_cluster_skills = pd.concat(new_features, axis=1)

In [ ]:
fe_cluster_skills

### Illustration Model

##### Create train/test matices

In [ ]:
combined_features_df = pd.concat([fe_cluster_skills,
                                 skills_ohe.droplevel(0,axis=1)],
                                 axis=1)
roles_df = ohe_df['DevType'].copy()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(combined_features_df, roles_df, test_size=0.3, random_state=0)

##### Compute samples weight to deal with classes imbalance

In [ ]:
# Assign class weight as an inverse of its frequency
class_weights = (1 / roles_df.sum(axis=1))

# Multiple class weight with the one-hot encoded value and get the mean of each sample
sample_weight = np.multiply(class_weights, Y_train.values).sum(axis=1)

##### Compute samples weight to deal with classes imbalance

In [ ]:
def f1_from_confusion_matrix(confusion_matrix):
    return (confusion_matrix[1,1] / 
            (confusion_matrix[1,1] + 
             (0.5 * (confusion_matrix[0,1] + confusion_matrix[1,0]))
             ))

In [ ]:
features_sets = {'original' : skills_ohe.droplevel(0,axis=1).columns.tolist(),
                 'clusters' : fe_cluster_skills.columns.tolist()}

In [ ]:
result = {}

for feature_set_name, features_set in features_sets.items():
    # Create sub trainings
    sub_train = X_train[features_sets].copy()
    sub_test = X_test[features_sets].copy()
    
    # Train classifier
    clf = MultiOutputClassifier(LogisticRegression())
    clf.fit(sub_train, Y_train, sample_weight=sample_weight)
    
    # Calculate F1 for training data
    multilabel_confusion_matricies = multilabel_confusion_matrix(Y_train, clf.predict(sub_train))
    f1_train_scores = [f1_from_confusion_matrix(matrix)
                       for matrix in multilabel_confusion_matricies]
    
    # Calculate F1 for test data
    multilabel_confusion_matricies = multilabel_confusion_matrix(Y_test, clf.predict(sub_test))
    f1_test_scores = [f1_from_confusion_matrix(matrix)
                       for matrix in multilabel_confusion_matricies]
    
    # Add to results
    set_result = pd.DataFrame({"train": f1_train_scores, "test": f1_test_scores},
                              index=roles_df.columns.to_list())
    results[feature_set_name] = set_result.sort_values('test')
    
    # Print
    print("Feature set: " + feature_set_name)
    print(".. Mean train F1: ", np.mean(f1_train_scores))
    print(".. Mean test F1: ", np.mean(f1_test_scores))

### Export new features

In [ ]:
features_path = os.path.join(EXPORT_FEATURES_DIR, "features_skills_clusters.pkl")
fe_cluster_skills.to_pickle(features_path)

In [ ]:
description_path = os.path.join(EXPORT_FEATURES_DIR, "features_skills_clusters_desscription.yaml")
with open(description_path, 'w') as outfile:
    ymal.dump(skills_clusters.to_dict(), outfile)